In [1]:
import pandas as pd
import numpy as np

In [2]:
class PCA(object):
    def __init__(self, data, n_component, corr=True):
        """
        传入的数据不必中心化与标准化，但是要以列为属性名
        """
        if not isinstance(data, pd.DataFrame):
            self.data = pd.DataFrame(data)
        else:
            self.data = data
        self.n_component = n_component
        self.corr = corr
        self.y_matrix = None
        self.conrate = None
        self.cov = None
        self.df = None

    def pca(self):
        if self.corr == True:
            self.data = (self.data-np.mean(self.data, axis=0)) / np.std(self.data,axis=0,ddof=1)
        else:
            self.data = self.data-np.mean(self.data, axis=0)# 列均值归0     
        self.cov = pd.DataFrame(np.cov(self.data.T))
        eig = np.linalg.eig(self.cov)
        eig_vector = pd.DataFrame(eig[1])
        self.df = pd.concat([eig_vector.T, pd.DataFrame(eig[0], columns=['eig'])], axis=1)
        tran_matrix = self.df.sort_values(by='eig',ascending=False).iloc[:self.n_component, :-1]
        self.y_matrix = np.dot(self.data, tran_matrix.T)
        return np.round(self.y_matrix, 2)

    def y2x(self, xi):
        vi = 0
        for j in range(self.n_component):
            vi = vi + self.df['eig'][j] * pow(self.df.iloc[j, xi-1],2)
        self.conrate = vi/self.cov.iloc[xi-1,xi-1]
        print("主成分对原有变量X{}保留信息比例为{:.2f}%".format(xi, 100*self.conrate))

In [3]:
data = pd.read_excel("data.xlsx").iloc[:,1:]

In [4]:
gdp = PCA(data, 3, corr=False)

In [5]:
gdp.pca()

array([[  686.21,  -161.69,   -10.08],
       [  678.62,   -94.99,    41.76],
       [  519.64,   -29.89,   -10.64],
       [  465.25,    41.93,   -21.6 ],
       [  358.52,    74.47,   -42.24],
       [  102.18,   197.25,   -72.11],
       [ -211.2 ,    76.84,   103.55],
       [ -603.29,    29.45,   115.21],
       [ -831.2 ,     9.67,   -38.72],
       [-1164.74,  -143.05,   -65.13]])

In [6]:
v_list = []
for i in range(1,9):
    gdp.y2x(i)
    v_list.append(gdp.conrate)
print("主成分对原有变量X{}保留信息比例最大，为{:.2f}%".format(1+v_list.index(max(v_list)), 100*max(v_list)))

主成分对原有变量X1保留信息比例为99.93%
主成分对原有变量X2保留信息比例为98.15%
主成分对原有变量X3保留信息比例为84.12%
主成分对原有变量X4保留信息比例为99.78%
主成分对原有变量X5保留信息比例为99.27%
主成分对原有变量X6保留信息比例为99.88%
主成分对原有变量X7保留信息比例为98.58%
主成分对原有变量X8保留信息比例为22.19%
主成分对原有变量X1保留信息比例最大，为99.93%
